# TASK 3 - STREAMING CIPHER

### 1. Define an object that, starting from a random bit generator, generates random bytes. 

In [11]:
from lfsr import LFSR, int_to_bin, bin_to_int
from itertools import islice
import random

class PseudoRandomByteGenerator(object):
    ''' class docstring '''

    def __init__(self, seed=None, bit_generator=None, **kwargs):

        """
        self.poly = kwargs['poly']
        self.seed = seed
        """

        """
        if self.seed == None: 
            self.seed = '0' * max(self.poly)
        while len(self.seed) < max(self.poly): 
            self.seed = '0' + self.seed
        """
        
        self.bit_generator = bit_generator
        if self.bit_generator == None:
            self.bits = []
            

            self.lfsr_inst = LFSR(kwargs['poly'], seed)

        elif self.bit_generator == 'random':
            random.seed(seed)
            self.bits = 0

    def __iter__(self):
        return self

    def bit_to_bytes(self, s):
        return int(s, 2).to_bytes(len(s) // 8, byteorder='big')
    
    def __next__(self):
        if self.bit_generator == None: 
            self.bits = self.lfsr_inst.run_steps(N=8)   #get a list of 8 bits represented as true and false
            self.bits = ''.join(['1' if b==True else '0' for b in self.bits])   #convert the list of bits to a string of 0 and 1
            byte = int(self.bits, 2)
        elif self.bit_generator == 'random':
            self.bits = random.getrandbits(8)
            byte = self.bits
        return byte

In [12]:
prbg = PseudoRandomByteGenerator(poly=[3, 1]) # iterator
digits = [byte for byte in islice(prbg, 10)]
print(bytes(digits))

b'\xe9\xd3\xa7N\x9d:t\xe9\xd3\xa7'


In [13]:
prbg = PseudoRandomByteGenerator(bit_generator='random') # iterator
digits = [byte for byte in islice(prbg, 10)]
print(bytes(digits)) # -> b'\x00x\xe7/\xdd7\xe1\\t\xc5'

b'D\x03$S:\xda\xab\x95\x0e\x12'


### 2. Define an object implementing a Stream Cipher that, given a Pseudo Random Byte Generator and a key, can encrypt and decrypt a message. 

In [12]:
class StreamCipher():
    ''' class docstring '''
    def __init__(self, key, prng=None, **kwargs):
        
        if prng is None:
            self.byte_gen = PseudoRandomByteGenerator(seed=key, **kwargs)
        else: 
            self.byte_gen = prng(key=key, **kwargs) 
    
    def encrypt(self, plaintext):
        N = len(plaintext)
        digits = [byte for byte in islice(self.byte_gen, N)]
        encrypted = [a ^ b for (a,b) in zip(bytes(digits), plaintext)]
        ciphertext = bytes(encrypted)
        return ciphertext
    
    def decrypt(self, ciphertext):
        N = len(ciphertext)
        digits = [byte for byte in islice(self.byte_gen, N)]
        decrypted = [a ^ b for (a,b) in zip(bytes(digits), ciphertext)]
        plaintext = bytes(encrypted)
        return plaintext

In [13]:
message = 'hello world!'
key = 0x0123456789ABCDEF
alice = StreamCipher(key, poly=[7, 1, 0])
plaintextA = message.encode('utf-8')
ciphertext = alice.encrypt(plaintextA)
bob = StreamCipher(key, poly=[7, 1, 0])
plaintextB = bob.encrypt(ciphertext)

print(plaintextA) # -> b'hello world!'
print(ciphertext) # -> b'\x9f\x03\xbcf\xfa\xdb`\xf6\x17\xce7\x88'
print(plaintextB) # -> b'hello world!'

b'hello world!'
b'\xf9\xc7\xdf\xa8\xba\xc6\x80\x93\xf5M\x1c\xf7'
b'hello world!'


### 3. Define an iterator that implements the A5/1 architecture and use it as bit generator in a Stream Cipher

In [20]:
class A5_1():

    def __init__(self, key, frame, verbose=0):

        self.lfsr1 = LFSR(poly=[19, 18, 17, 14, 0], state=0)
        self.lfsr2 = LFSR(poly=[22, 21, 0], state=0)
        self.lfsr3 = LFSR(poly=[23, 22, 21, 8, 0], state=0)
        self.counter = 0
        self.verbose = verbose

        if len(bin(key)[2:]) > 64: 
            raise Exception("The key must be 64 bit")
        else: 
            self.key = int_to_bin(key, 64)[::-1]
        
        if len(bin(frame)[2:])>22:
            raise Exception("The frame must be 22 bit")
        else: 
            self.frame = int_to_bin(frame, 22)[::-1]
        
        self.key_ins()
        self.frame_ins()
        self.key_mixing()

    def key_ins(self):
        for count in range(64):
            #feedback is None
            self.lfsr1.feedback, self.lfsr3.feedback, self.lfsr3.feedback = self.key[count]^self.lfsr1.feedback, self.key[count]^self.lfsr2.feedback, self.key[count]^self.lfsr3.feedback
            if self.verbose == 1:
                if count == 0: 
                    print('{:>2} {:>3} {:>10} {:>12} {:>12} {:>10}'.format('iter', 'key', 'LFSR 1', 'LFSR2', 'LFSR3', 'out'))
                self.print_a5(count+1, mode='key')

    def frame_ins(self):
        for count in range(64):
            self.lfsr1.feedback, self.lfsr3.feedback, self.lfsr3.feedback = self.frame[count]^self.lfsr1.feedback, self.frame[count]^self.lfsr2.feedback, self.frame[count]^self.lfsr3.feedback
            self.__next__(key, frame=1)
            if self.verbose == 1: 
                if count == 0: 
                    print('{:>2} {:>3} {:>10} {:>12} {:>12} {:>10}'.format('iter', 'key', 'LFSR 1', 'LFSR2', 'LFSR3', 'out'))
                self.print_a5(count+1, mode='frame')
    
    def key_mixing(self):
        for count in range(100):
            self.__next__(key, frame=0)
            if self.verbose == 1: 
                if count == 0: 
                    print('{:>2} {:>3} {:>10} {:>12} {:>12} {:>10}'.format('iter', 'key', 'LFSR 1', 'LFSR2', 'LFSR3', 'out'))
                self.print_a5(count+1, mode='normal')
    
    def print_a5(self, count, mode): 
        state1 = bool_list_hex(self.lfsr1.state[2:])
        state2 = bool_list_hex(self.lfsr2.state[2:])
        state3 = bool_list_hex(self.lfsr3.state[2:])
        if mode=='key':
            print('{:>2} {:>3} {:>12} {:>12} {:>12} {:>12}'.format(count, self.key[count-1], state1, state2, state3, self.out))
        elif mode == 'frame':
                print('{:>2} {:>3} {:>12} {:>12} {:>12} {:>12}'.format(count, self.frame[count-1], state1, state2, state3, self.out))
        elif mode == 'normal':
                print('{:>2} {:>3} {:>12} {:>12} {:>12} {:>12}'.format(count, state1, state2, state3, self.majority, self.out))

    def __iter__(self):
        return self
    
    def __next__(self, key_frame=0):

        if key_frame == 0:
            self.majority = not(floor(1/2+(int(self.lfsr1.state[8])+int(self.lfsr2.state[10])+int(self.lfsr3.state[10])-1/2)/3))
            if self.majority ^ self.lfsr1.state[8]:
                next(self.lfsr1)
            if self.majority ^ self.lfsr2.state[10]:
                next(self.lfsr2)
            if self.majority ^ self.lfsr3.state[8]:
                next(self.lfsr3)
        else: 
            next(self.lfsr1)
            next(self.lfsr2)
            next(self.lfsr3)
        self.out = self.lfsr1.output ^ self.lfsr2.output ^ self.lfsr3.output
        return int(self.out)


In [21]:
key, frame = 0x0123456789ABCDEF, 0x2F695A
a5 = A5_1(key, frame, verbose=1)

TypeError: unsupported operand type(s) for ^: 'int' and 'NoneType'

In [ ]:
message = 'hello world!'
key = 0x0123456789ABCDEF

# create a StreamCipher instance for Alice and Bob
# no PRNG is specified, then LFSR is employed
alice = StreamCipher(key)
bob = StreamCipher(key)

plaintextA = message.encode('utf-8') # string to bytes
ciphertext = alice.encrypt(plaintextA) # encryption by Alice
plaintextB = bob.decrypt(ciphertext) # decryption by Bob

print(plaintextA) # -> b'hello world!'
print(ciphertext) # -> b'\x9f\x03\xbcf\xfa\xdb`\xf6\x17\xce7\x88'
print(plaintextB) # -> b'hello world!'

### 4. Define an iterator that implements the RC4 architecture and use it in the Stream Cipher object to encrypt/decrypt a message

In [ ]:
class RC4(object): 
    def __init__(self, key, drop):
        self.key = lisy(key)
        self.P = []
        self.KSA()
        
        for i in range(drop):
            self.__next__()

    def KSA(self):
        j = 0 
        L = len(self.key)
        for i in range(265):
            self.P.append(i)
        
        for i in range(265):
            print('{:3d}{:3d}{:3d}{:3d}'.format(i, j, self.P[i], self.P[j]))
            j = (j + self.P[i] + self.key[i % L]) % 265
            self.P[i], self.P[j] = self.P[j], self.P[i]
        
    def __iter__(self):
        return self
    
    def __next__(self):
        i = 0
        j = 0
        i = (i+1) % 265
        j = (j+self.P[i]) % 265
        self.P[i], self.P[j] = self.P[j], self.P[i]
        self.K = self.P[(self.P[i]+self.P[j]) % 265]
        return self.K

In [ ]:
message = 'hello world!’
key = b'0123456789ABCDEF'

# create a StreamCipher instance for both Alice and Bob
alice = StreamCipher(key, prng=RC4, drop=10)
bob = StreamCipher(key, prng=RC4, drop=10)

plaintextA = message.encode('utf-8') # -> b'Hello world!'
ciphertext = alice.encrypt(plaintextA) # -> b'/\x9e\xf9\x83@\x81}\xa9\xd0\xd4\xd5\xf4'
plaintextB = bob.decrypt(ciphertext) # -> b'Hello world!'